## Library

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import albumentations as A
from torch.utils.data import DataLoader, Dataset
import os
import cv2
from albumentations.pytorch import ToTensorV2
import torch.nn.functional as F
import math
from torch.nn import Parameter
import torch.optim as optim
import torchvision.transforms as transforms
import time
import torch.nn as nn
from torch.nn import Linear, Conv2d, BatchNorm1d, BatchNorm2d, ReLU, Dropout, MaxPool2d, Sequential, Module
from PIL import Image

## Test data

In [ ]:
# test folder path 

test_path = "data/test"

In [ ]:
#Count - Check

folder = os.listdir(test_path)
count = 0
print(len(folder))
for i in folder:
  path = os.path.join(test_path, i)
  num_images = len(os.listdir(path))
  count = count + num_images
print(count)

## Load Backbone

In [ ]:
INPUT_SIZE = (112,112)

In [ ]:
from backbone.model_irse import IR_SE_50
BACKBONE = IR_SE_50(INPUT_SIZE)

## Transforms

In [ ]:
import torchvision.transforms as transforms

input_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize([int(112), int(112)]),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])

## Test function

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image

def TEST(folder_path, model, transforms):

    label_org = []
    dir_org = []
    label_test = []
    dir_test_path = []

    # Put the model in evaluation mode
    model.eval()

    # Disable gradient calculation
    with torch.no_grad():
        # Iterate over each subfolder in the folder_path
        for label_index, subfolder_name in enumerate(os.listdir(folder_path)):
            subfolder_path = os.path.join(folder_path, subfolder_name)
            image_files = os.listdir(subfolder_path)

            for image_index, image_file in enumerate(image_files):

                image_path = os.path.join(subfolder_path, image_file)

                if "ORG_EM" in image_path:

                    image = Image.open(image_path).convert('RGB')
                    image = transforms(image)
                    # Extract the embedding for the first image in the folder
                    embedding = model(image.unsqueeze(0).to("cuda"))
                    dir_org.append(embedding)
                    label_org.append(label_index)
                else:
                    # Store the path and label for other images
                    dir_test_path.append(image_path)
                    label_test.append(label_index)

        predict_label = []

        # Iterate over test images
        for test_image_path in dir_test_path:

            test_image = Image.open(test_image_path).convert('RGB')
            test_image = transforms(test_image)

            # Extract the embedding for the test image
            test_embedding = model(test_image.unsqueeze(0).to("cuda"))
            similarities = []

            # Calculate cosine similarity with each original embedding
            for org_embedding in dir_org:

                cosine_sim = cosine_similarity(org_embedding.cpu().detach().numpy(), test_embedding.cpu().detach().numpy())
                similarities.append(cosine_sim[0][0])

            max_similarity_index = np.argmax(similarities)

            predict_label.append(label_org[max_similarity_index])

        # Calculate accuracy
        accuracy = accuracy_score(predict_label, label_test)
        print(f'----Accuracy: {accuracy:.4f}')

In [ ]:
for i in range(10, 111, 10):
  print(f'model {i}:')
  model = torch.load(f"checkpoint/ISE_{i}.pth").to("cuda")
  TEST(test_path, model, input_transforms)